# Load LoRA adapters

In [1]:
import torch
if torch.cuda.get_device_capability() < (7, 5):
    raise ValueError(f"You got a GPU with capability {torch.cuda.get_device_capability()}, need at least (7, 5)")
else: print("OK")

OK


In [2]:
%pip install bitsandbytes==0.37.0 transformers==4.27.4 datasets==2.7.0 accelerate==0.18.0 loralib==0.1.1 peft==0.3.0.dev0

Note: you may need to restart the kernel to use updated packages.


#### Load base model

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

CACHE_DIR = '/media/tfsservices/DATA/NLP/cache/'
# MODEL_NAME = "facebook/opt-6.7b"
MODEL_NAME = "facebook/opt-30b"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,        # bitsandbytes lib required (convert the loaded model into mixed-8bit quantized model.)
    device_map='auto',
    torch_dtype=torch.float16,
    cache_dir=CACHE_DIR)      # path to a directory in which a downloaded pretrained model
    # low_cpu_mem_usage=True,   # loads the model using ~1x model size CPU memory
    # offload_state_dict=True)  # temporarily offload the CPU state dict to the hard drive to avoid getting out of CPU RAM
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


/home/tfsservices/miniconda3/envs/nlp_test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:36<00:00,  5.16s/it]


In [4]:
def test_generate_marketing():

    inputs = tokenizer("Hi {FirstName}", return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=200)
        print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [5]:
model.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 7168, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 7168)
      (final_layer_norm): LayerNorm((7168,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-47): 48 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear8bitLt(in_features=7168, out_features=7168, bias=True)
            (v_proj): Linear8bitLt(in_features=7168, out_features=7168, bias=True)
            (q_proj): Linear8bitLt(in_features=7168, out_features=7168, bias=True)
            (out_proj): Linear8bitLt(in_features=7168, out_features=7168, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((7168,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear8bitLt(in_features=7168, out_features=28672, bias=True)
          (fc2): Linear8bitLt(in_features=28672, out_features=7168, bias=True

In [6]:
test_generate_marketing()

Hi {FirstName}!

Look out for our weekly newsletters that feature the latest news in and around your community.

We'll feature local events, announcements, and other information that you won't find anywhere else.

All you have to do is sign up below.

Email Address

First Name

Last Name

By submitting this form, you are granting: The Community Foundation of Greater New Haven, P.O. Box 609, New Haven, Connecticut, 06511, United States, http://www.cfgnh.org permission to email you. You may unsubscribe via the link found at the bottom of every email. (See our Email Privacy Policy for details.) Emails are serviced by Constant Contact.


#### Load and apply LoRA adapter

In [7]:
from peft import PeftModel, PeftConfig

# peft_model_id = "lora_adapters"
# peft_model_id = "lora_adapters-13b"
peft_model_id = "lora_adapters-30b"

config = PeftConfig.from_pretrained(peft_model_id)
model = PeftModel.from_pretrained(model, peft_model_id)


In [9]:
test_generate_marketing()

Hi {FirstName}

I'm reaching out because we discussed {MakeModel} Generators before. I noticed that you have a need for a {MakeModel} Generators and I have a {MakeModel} Generators for sale at the moment. The unit is complete and available immediately. The seller is looking to get back 100% of the purchase price.

Please review the details in the link below and let me know if you are interested.

{Listing}

{SendersSignature}

{SendersSignature}

{Listing}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{SendersSignature}

{
